<a href="https://colab.research.google.com/github/subashmb1998/Project_HKS/blob/main/cutomers_table_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!rm spark-3.1.2-bin-hadoop2.7.tgz

# Install findspark
!pip install -q findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# Initialize findspark
import findspark
findspark.init()

# Verify the installation
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("colab").getOrCreate()
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#using delimiter to confirm file separated by ',' coma
#using the option multiline 'true'
#https://chatgpt.com/share/cd74468d-506f-452d-9970-dbe2a757abc4

df_read = spark.read.format("csv").option("header","true").option("delimiter",',').option('multiline','true').load("/content/drive/MyDrive/Projects/HKS/Customers Vaniamkulam on 23 05 2024.csv")

In [ ]:
df_read.show(1)

+---------------+-------------+-----------+--------------------+-------------+-------------+------------+---------------+----------------+-----------+-------------+-------------+----------------+--------------------+-------------------+---------------------------+-------------------+-----------+---------------------+------------------+------------------+--------------------+-----+----------+----------+--------------------+--------------------+------------------------------+------------------------------+-------------+------------------+----------------------+---------+---------+----------+-------------+---------------+--------------+------------------+--------------------+-------------+---------------------+----------------+-----------------------+---------------------+-------------------+-----------------+-------------------+------------------+---------+---------------+
|CUSTOMER_NUMBER|CUSTOMER_NAME|    QR_CODE|        ORGANIZATION|SURVEYOR_NAME|         WARD|TRADING_TYPE|PROPERTY_NU

In [ ]:
df_read.columns

['CUSTOMER_NUMBER',
 'CUSTOMER_NAME',
 'QR_CODE',
 'ORGANIZATION',
 'SURVEYOR_NAME',
 'WARD',
 'TRADING_TYPE',
 'PROPERTY_NUMBER',
 'ASSOCIATION_NAME',
 'DOOR_STATUS',
 'PROPERTY_NAME',
 'PROPERTY_TYPE',
 'BIO_WASTE_STATUS',
 'NON_BIO_WASTE_STATUS',
 'TERRACE_FARM_STATUS',
 'PROPERTY_OWNER_PHONE_NUMBER',
 'PROPERTY_OWNER_NAME',
 'TOTAL_HOUSE',
 'GREEN_PROTOCOL_STATUS',
 'ASSOCIATION_NUMBER',
 'PROPERTY_OWNERSHIP',
 'ADDRESS',
 'IMAGE',
 'LATITUDE',
 'LONGITUDE',
 'FORMATTED ADDRESS',
 'PROPERTY_TENANT_NAME',
 'PROPERTY_TENANT_PHONE_NUMBER27',
 'PROPERTY_TENANT_PHONE_NUMBER28',
 'PROPERTY_AREA',
 'BIO_WASTE_QUANTITY',
 'NON_BIO_WASTE_QUANTITY',
 'ROAD_NAME',
 'ROAD_TYPE',
 'ROAD_WIDTH',
 'TOTAL_MEMBERS',
 'TOTAL_GROWN_UPS',
 'TOTAL_CHILDREN',
 'RATION_CARD_NUMBER',
 'RATION_CARD_TYPE',
 'ANNUAL_INCOME',
 'BIO_WASTE_MANURE_USED',
 'BIO_WASTE_REPAIR',
 'REQUIRE_DETAILED_SURVEY',
 'VEHICLE_ACCESSIBILITY',
 'TOILET_WASTE_SYSTEM',
 'GRAY_WATER_SYSTEM',
 'SEPTIC_TANK_REMOVAL',
 'WASTE_WATER_S

In [ ]:
from pyspark.sql.functions import *

In [ ]:
#In text data, especially addresses, it's common to have multiple lines,
#especially if the data was manually entered or extracted from a multi-line source.
#Newline and carriage return characters can break the consistency of the address format or cause issues when processing the data
#so here using regexp_replace
#regexp_replace: This is a function from the pyspark.sql.functions module that replaces substrings in a string column based on a regular expression pattern.


df_read = df_read.withColumn("ADDRESS", regexp_replace('ADDRESS','[\b\n\r]',','))

In [ ]:
cols = [
 'CUSTOMER_NUMBER',
 'CUSTOMER_NAME',
 'QR_CODE',
 'ORGANIZATION',
 'SURVEYOR_NAME',
 'WARD',
 'TRADING_TYPE',
 'PROPERTY_NUMBER',
 'ASSOCIATION_NAME',
 'DOOR_STATUS',
 'PROPERTY_NAME',
 'PROPERTY_TYPE',
 'BIO_WASTE_STATUS',
 'NON_BIO_WASTE_STATUS',
 'TERRACE_FARM_STATUS',
 'PROPERTY_OWNER_NAME',
 'TOTAL_HOUSE',
 'GREEN_PROTOCOL_STATUS',
 'ASSOCIATION_NUMBER',
 'PROPERTY_OWNERSHIP',
 'ADDRESS',
 'IMAGE',
 'LATITUDE',
 'LONGITUDE',
 'FORMATTED ADDRESS',
 'PROPERTY_TENANT_NAME',
 'PROPERTY_TENANT_PHONE_NUMBER28',
 'PROPERTY_AREA',
 'BIO_WASTE_QUANTITY',
 'NON_BIO_WASTE_QUANTITY',
 'ROAD_NAME',
 'ROAD_TYPE',
 'ROAD_WIDTH',
 'TOTAL_MEMBERS',
 'TOTAL_GROWN_UPS',
 'TOTAL_CHILDREN',
 'RATION_CARD_NUMBER',
 'RATION_CARD_TYPE',
 'ANNUAL_INCOME',
 'BIO_WASTE_MANURE_USED',
 'BIO_WASTE_REPAIR',
 'REQUIRE_DETAILED_SURVEY',
 'VEHICLE_ACCESSIBILITY',
 'TOILET_WASTE_SYSTEM',
 'GRAY_WATER_SYSTEM',
 'SEPTIC_TANK_REMOVAL',
 'WASTE_WATER_SYSTEM',
 'QR_NUMBER',
 'PROPERTY_LENGTH'
]

In [ ]:
df_new = df_read.select(cols)

In [ ]:
df_new.createOrReplaceTempView("df_new")

In [ ]:
# Get the list of columns
column_names = df_new.columns

# Identify columns where all values are null
null_columns = [col for col in column_names if df_new.filter(df_new[col].isNotNull()).count() == 0]

# Print null columns
print("Columns with all null values:", null_columns)


Columns with all null values: ['ASSOCIATION_NAME', 'BIO_WASTE_STATUS', 'TERRACE_FARM_STATUS', 'GREEN_PROTOCOL_STATUS', 'ASSOCIATION_NUMBER', 'IMAGE', 'PROPERTY_AREA', 'BIO_WASTE_QUANTITY', 'NON_BIO_WASTE_QUANTITY', 'TOTAL_MEMBERS', 'TOTAL_GROWN_UPS', 'TOTAL_CHILDREN', 'ANNUAL_INCOME', 'BIO_WASTE_MANURE_USED', 'BIO_WASTE_REPAIR', 'TOILET_WASTE_SYSTEM', 'GRAY_WATER_SYSTEM', 'SEPTIC_TANK_REMOVAL', 'PROPERTY_LENGTH']


In [ ]:
# Select columns that are not completely null
non_null_columns = [col for col in column_names if col not in null_columns]

# Create a new DataFrame with non-null columns
df_cleaned = df_new.select(*non_null_columns)

In [ ]:
#df_cleaned.createOrReplaceTempView("df_cleaned_new")

In [ ]:
#spark.sql("SELECT * FROM df_cleaned WHERE PROPERTY_NUMBER NOT LIKE '%00%'").show()

In [ ]:
#spark.sql("select distinct WASTE_WATER_SYSTEM from df_cleaned_new ").show()

In [ ]:
df_cleaned.select("WASTE_WATER_SYSTEM").distinct().show()

+--------------------+
|  WASTE_WATER_SYSTEM|
+--------------------+
|                null|
|Septic tank and s...|
+--------------------+



In [ ]:
#spark.sql("select distinct NON_BIO_WASTE_STATUS from df_cleaned_new ").show()

In [ ]:
df_cleaned.select("NON_BIO_WASTE_STATUS").distinct().show()

+--------------------+
|NON_BIO_WASTE_STATUS|
+--------------------+
|                null|
|                  No|
+--------------------+



In [ ]:
#after checking some columns have unmatched values so replacing with similar value


df_clean1 = df_cleaned.na.fill({"NON_BIO_WASTE_STATUS":"No"})

In [ ]:
df_clean2= df_clean1.na.fill({"TOTAL_HOUSE":"1"})

In [ ]:
df_clean3= df_clean2.na.fill({"REQUIRE_DETAILED_SURVEY":"No"})

In [ ]:
df_clean4 = df_clean3.replace("0","null",subset=["RATION_CARD_NUMBER"])

In [ ]:
#droping unwamted coloumns
df_clean5 = df_clean4.drop("ROAD_NAME","ROAD_TYPE","ROAD_WIDTH","QR_NUMBER","NON_BIO_WASTE_STATUS","REQUIRE_DETAILED_SURVEY","VEHICLE_ACCESSIBILITY","WASTE_WATER_SYSTEM","FORMATTED ADDRESS")

In [ ]:
month_to_number = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
    "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

In [ ]:
#in property_number column some numbers read as month so replace it with number


df_date_clean = df_clean5
for month, numb in month_to_number.items():
    df_date_clean = df_date_clean.withColumn("PROPERTY_NUMBER",
                                             regexp_replace(col("PROPERTY_NUMBER"), month, numb))

In [ ]:
df_date_clean.show()

+---------------+----------------+-----------+--------------------+--------------+-----------------+-----------------+---------------+-----------+--------------------+-------------+-------------------+-----------+------------------+--------------------+------------+------------+--------------------+------------------------------+------------------+--------------------+
|CUSTOMER_NUMBER|   CUSTOMER_NAME|    QR_CODE|        ORGANIZATION| SURVEYOR_NAME|             WARD|     TRADING_TYPE|PROPERTY_NUMBER|DOOR_STATUS|       PROPERTY_NAME|PROPERTY_TYPE|PROPERTY_OWNER_NAME|TOTAL_HOUSE|PROPERTY_OWNERSHIP|             ADDRESS|    LATITUDE|   LONGITUDE|PROPERTY_TENANT_NAME|PROPERTY_TENANT_PHONE_NUMBER28|RATION_CARD_NUMBER|    RATION_CARD_TYPE|
+---------------+----------------+-----------+--------------------+--------------+-----------------+-----------------+---------------+-----------+--------------------+-------------+-------------------+-----------+------------------+--------------------+---

In [ ]:
#df_clean6 = df_clean5.replace("%[A-Za-z]%","0",subset=["PROPERTY_NUMBER"])

In [ ]:
df_clean7 = df_date_clean.withColumn("PROPERTY_NUMBER", regexp_replace("PROPERTY_NUMBER", "-", "/"))

In [ ]:
#df_clean7.createOrReplaceTempView("df_clean7")
#df_clean8 = spark.sql("SELECT * FROM df_clean7 WHERE PROPERTY_NUMBER NOT LIKE '%R%'")
#df_clean8.createOrReplaceTempView("df_clean8_view")

In [ ]:
df_clean10 = df_clean7.sort("WARD")

In [ ]:
#The code snippet creates a new DataFrame df_clean11 from df_clean7, adding or modifying the PROPERTY_NUMBER column based on a condition.
#Specifically, it checks the length of the value in the PROPERTY_NUMBER column.
#If the length of PROPERTY_NUMBER is exactly 1, it replaces that value with 'X'.
#If the length is not 1, it retains the original value in the PROPERTY_NUMBER column.
#This is achieved using the when and otherwise functions from PySpark, which allow conditional transformations on DataFrame columns.


df_clean11 = df_clean10.withColumn(
    "PROPERTY_NUMBER",
    when(length(col("PROPERTY_NUMBER"))==1, 'X')
    .otherwise(col("PROPERTY_NUMBER"))
)

In [ ]:
#df_clean12 = df_clean10.filter(df_clean11.PROPERTY_NUMBER == 'X')

In [ ]:
df_clean13 = df_clean11.filter(df_clean11.PROPERTY_NUMBER != 'X')

In [ ]:
#df_clean13.createOrReplaceTempView("df_clean13_view")
#df_clean14 = spark.sql("SELECT * FROM df_clean13_view WHERE WARD IS NOT NULL")

In [ ]:
df_clean14 = df_clean13.filter(col("WARD").isNotNull())

In [ ]:
df_clean14.show()

+---------------+------------------+-----------+--------------------+-------------+---------+-----------------+---------------+-----------+--------------------+--------------------+-------------------+-----------+------------------+--------------------+------------+------------+--------------------+------------------------------+------------------+--------------------+
|CUSTOMER_NUMBER|     CUSTOMER_NAME|    QR_CODE|        ORGANIZATION|SURVEYOR_NAME|     WARD|     TRADING_TYPE|PROPERTY_NUMBER|DOOR_STATUS|       PROPERTY_NAME|       PROPERTY_TYPE|PROPERTY_OWNER_NAME|TOTAL_HOUSE|PROPERTY_OWNERSHIP|             ADDRESS|    LATITUDE|   LONGITUDE|PROPERTY_TENANT_NAME|PROPERTY_TENANT_PHONE_NUMBER28|RATION_CARD_NUMBER|    RATION_CARD_TYPE|
+---------------+------------------+-----------+--------------------+-------------+---------+-----------------+---------------+-----------+--------------------+--------------------+-------------------+-----------+------------------+--------------------+---

In [ ]:
df_clean14.select("PROPERTY_TYPE").distinct().show()

+--------------------+
|       PROPERTY_TYPE|
+--------------------+
|       Public Toilet|
|Health Care Insti...|
|              Office|
|                Flat|
|                Shop|
|Poultry/ Slaughte...|
|Religious Institu...|
|        Saloon / Spa|
|               House|
|       Lodge/ Hostel|
| Hotel & Food Courts|
|Educational Insti...|
|          Auditorium|
+--------------------+



In [ ]:
customers_final = df_clean14.withColumn("FEE_AMNT", when(df_clean14.PROPERTY_TYPE == 'House', 50).otherwise('100'))

In [ ]:
#customers_final.repartition(1).write.mode("overwrite").option("header","true").csv(f"/content/drive/MyDrive/Projects/HKS/customers_final.csv")

In [ ]:
customers_final.repartition(1) \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .option("path", "/content/drive/MyDrive/Projects/HKS/customers_final") \
    .save()